In [ ]:
!pip install numpy
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd
import math

class MLP():
  
  def __init__(self, vec_x, numClass):
    # This can be considered the input layer
    # It is a vector that stores the initial values from an array
    self.inputVec = vec_x
    self.outClasses = numClass

  class layer():
    ### Start of the layer class 

    def __init__(self, nodes, activation):
      self.__numNodes = nodes
      self.__actFunc = activation

    class node():
      ### Start of the node class

      def __init__(self, inputVec, inputWeight):
        self.__sumVal = 0
        self.__outVal = 0
        self.__actFun = ""
        self.__vector = inputVec
        self.__weights = inputWeight
        self.__bias = np.random.random()
      
      def setSum(self, sum = 0): 
        for (i, j) in list(zip(self.__vector, self.__weights)):
          sum = sum + (i * j)
        self.__sumVal = sum + self.__bias
        # Function does not return anything but sets the value of the sum

      def activationFun(self, choice):
        if (choice == "sigmoid"):
          self.__outVal = (1 / (1 + pow(math.e, -1*self.sumVal)))
        elif (choice == "relu"):
          self.__outVal = (max(0.1*self.nodeSum, self.sumVal))
        
        # Function returns the value of sum after going through the activation function
        return self.__outVal
    
    def addLayer(self, inputVector):
      # Randomize the weights
      layerWeights = np.array([np.random.random() for i in range(self.__numNodes)])

      # Instantiate the number of nodes based on numNodes
      layerNodes = [self.node(inputVector, layerWeights).activationFun(self.__actFunc) for i in range(self.__numNodes)]
      # Now I have a layer that is a list of values

      # Now return it as an np array
      return np.array(layerNodes)

  def activateModel(self):
    # Simulate the hidden layer that takes the input layer
    hidden1 = self.layer(10, "relu")
    lastHidden = hidden1.addLayer(self.inputVec)
    modelPredVal, modelPredClass = self.outputLayer(lastHidden)
    

  def outputLayer(self, lastHidden):
    outLayer = self.layer(self.outClasses, "sigmoid")
    return max(outLayer), outLayer.index(max(outLayer))


In [ ]:
model1 = MLP()